## Imports

In [ ]:
# pip install --upgrade scikit-learn

In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.4
Sklearn  0.24.1


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [2]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../Datasets/Tabular/titanic/"

## Load data

In [3]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [4]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [5]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [6]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = lambda x : x.split(',')[1].split('.')[0].strip()

# YOUR CODE HERE

# df2 = df.assign(Title = lambda name: ((df['Name']).split(',').split('.')))
# df = df.assign(Title = lambda x: (x['Name'].str.split('.')))
# df = df.assign(Title = lambda x: (x['Name'].str.split('.')))
# df["Title"] = df['Name'].apply(lambda x : x.split(',')[1].split('.')[0].strip())
# df = df.assign(Title = lambda x: (x['Name'].str.split(',').str.split('.')))

# title_np = np.array(df['Name'].str.split(','))
# for i in title_np:
#     del i[0]
# print(title_np)
# my_np =np.array(df["Title"])
# my_list = list(my_np)
# my_list = my_list.split('.')
# my_list2 = []


# print(df)


# raise NotImplementedError()

df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)
df.head(5)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [7]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [8]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [9]:
# Use map to apply the prevous dict

df["Title"] =  df['Title'].map(title_dictionary)
df_test["Title"] = df_test['Title'].map(title_dictionary)
print(len(df["Title"]))
print(len(df_test["Title"]))

# YOUR CODE HERE
# raise NotImplementedError()

891
418


In [10]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [11]:
print(df.head(5))

             Survived  Pclass  \
PassengerId                     
1                   0       3   
2                   1       1   
3                   1       3   
4                   1       1   
5                   0       3   

                                                          Name     Sex   Age  \
PassengerId                                                                    
1                                      Braund, Mr. Owen Harris    male  22.0   
2            Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
3                                       Heikkinen, Miss. Laina  female  26.0   
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
5                                     Allen, Mr. William Henry    male  35.0   

             SibSp  Parch            Ticket     Fare Cabin Embarked Title  
PassengerId                                                                
1                1      0         A/5 21171   7.2500   

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [12]:
df_test['Cabin']

PassengerId
892      NaN
893      NaN
894      NaN
895      NaN
896      NaN
        ... 
1305     NaN
1306    C105
1307     NaN
1308     NaN
1309     NaN
Name: Cabin, Length: 418, dtype: object

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [13]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [16]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [31]:
"""
num_preprocessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
])

cat_preporcessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
  # Some Encoder here. Remember to handle_unknown
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro
""";

# YOUR CODE HERE
num_4_treeModels = pipeline.Pipeline(steps=[('imputer', impute.SimpleImputer(strategy= 'mean', add_indicator=False)),
                                             ('scaler', preprocessing.StandardScaler())])
cat_4_treeModels = pipeline.Pipeline(steps= [('imputer', impute.SimpleImputer(strategy= 'constant', fill_value='missing')),
                                             ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))])
tree_prepro = compose.ColumnTransformer(transformers=[('num', num_4_treeModels, num_vars),
                                                ('cat', cat_4_treeModels, cat_vars)], remainder='drop')


# print(type(tree_prepro))
# print(type(num_4_treeModels))
# print(type(cat_4_treeModels))
# print(len(num_4_treeModels))
# print(len(cat_4_treeModels))
# raise NotImplementedError()
tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

In [18]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

AssertionError: 

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [19]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [26]:
"""
tree_classifiers = {
  "Decision Tree":
  "Extra Trees": 
  "Random Forest":
  "AdaBoost":
  "Skl GBM":enable_hist_gradient_boosting(),
  "Skl HistGBM":
  "XGBoost":
  "LightGBM":
  "CatBoost":
tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}
tree_classifiers["LightGBM"]
""";


# YOUR CODE HERE
# raise NotImplementedError()
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees": ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":HistGradientBoostingClassifier(),#enable_hist_gradient_boosting(),
  "Skl HistGBM":HistGradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [23]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [32]:
"""
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    # CODE HERE
)
"""
# YOUR CODE HERE
# raise NotImplementedError()

x_train, x_val, y_train, y_val = model_selection.train_test_split(x, y, test_size = 0.2, stratify = y, random_state = 4)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    
    # GET PREDICTIONS USING x_val
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""

# YOUR CODE HERE
# raise NotImplementedError()

for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
#     full_model = pipeline.Pipeline([('preprocessor', tree_prepro),('model',model_name)])
    model.fit(x_train,y_train)
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    
    # GET PREDICTIONS USING x_val
#     pred = full_model.predict(x_val)# CODE HERE
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

C:\Users\Study1\anaconda3\envs\Stv Machine Learning\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:07:45] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6874585	total: 229ms	remaining: 3m 48s
1:	learn: 0.6817921	total: 232ms	remaining: 1m 55s
2:	learn: 0.6768098	total: 234ms	remaining: 1m 17s
3:	learn: 0.6720448	total: 237ms	remaining: 59s
4:	learn: 0.6669187	total: 239ms	remaining: 47.6s
5:	learn: 0.6622295	total: 242ms	remaining: 40.1s
6:	learn: 0.6573380	total: 244ms	remaining: 34.6s
7:	learn: 0.6524357	total: 247ms	remaining: 30.6s
8:	learn: 0.6468926	total: 249ms	remaining: 27.4s
9:	learn: 0.6418077	total: 252ms	remaining: 24.9s
10:	learn: 0.6366277	total: 254ms	remaining: 22.9s
11:	learn: 0.6317214	total: 257ms	remaining: 21.1s
12:	learn: 0.6279207	total: 259ms	remaining: 19.7s
13:	learn: 0.6242446	total: 261ms	remaining: 18.4s
14:	learn: 0.

196:	learn: 0.3889898	total: 557ms	remaining: 2.27s
197:	learn: 0.3885964	total: 560ms	remaining: 2.27s
198:	learn: 0.3882224	total: 562ms	remaining: 2.26s
199:	learn: 0.3878513	total: 563ms	remaining: 2.25s
200:	learn: 0.3874993	total: 565ms	remaining: 2.25s
201:	learn: 0.3874447	total: 566ms	remaining: 2.24s
202:	learn: 0.3870912	total: 568ms	remaining: 2.23s
203:	learn: 0.3866994	total: 570ms	remaining: 2.22s
204:	learn: 0.3862415	total: 572ms	remaining: 2.22s
205:	learn: 0.3858974	total: 574ms	remaining: 2.21s
206:	learn: 0.3856541	total: 576ms	remaining: 2.21s
207:	learn: 0.3856045	total: 577ms	remaining: 2.2s
208:	learn: 0.3853270	total: 579ms	remaining: 2.19s
209:	learn: 0.3849022	total: 581ms	remaining: 2.19s
210:	learn: 0.3845265	total: 583ms	remaining: 2.18s
211:	learn: 0.3841572	total: 585ms	remaining: 2.17s
212:	learn: 0.3838030	total: 588ms	remaining: 2.17s
213:	learn: 0.3833648	total: 590ms	remaining: 2.17s
214:	learn: 0.3831301	total: 592ms	remaining: 2.16s
215:	learn: 0

370:	learn: 0.3526060	total: 895ms	remaining: 1.52s
371:	learn: 0.3525111	total: 897ms	remaining: 1.51s
372:	learn: 0.3524957	total: 898ms	remaining: 1.51s
373:	learn: 0.3524125	total: 901ms	remaining: 1.51s
374:	learn: 0.3522401	total: 902ms	remaining: 1.5s
375:	learn: 0.3520523	total: 904ms	remaining: 1.5s
376:	learn: 0.3518778	total: 906ms	remaining: 1.5s
377:	learn: 0.3517669	total: 908ms	remaining: 1.49s
378:	learn: 0.3516747	total: 910ms	remaining: 1.49s
379:	learn: 0.3515240	total: 912ms	remaining: 1.49s
380:	learn: 0.3513903	total: 914ms	remaining: 1.48s
381:	learn: 0.3513086	total: 916ms	remaining: 1.48s
382:	learn: 0.3512521	total: 918ms	remaining: 1.48s
383:	learn: 0.3511073	total: 920ms	remaining: 1.48s
384:	learn: 0.3509698	total: 922ms	remaining: 1.47s
385:	learn: 0.3508666	total: 924ms	remaining: 1.47s
386:	learn: 0.3506690	total: 927ms	remaining: 1.47s
387:	learn: 0.3505329	total: 929ms	remaining: 1.46s
388:	learn: 0.3504412	total: 931ms	remaining: 1.46s
389:	learn: 0.3

546:	learn: 0.3305845	total: 1.23s	remaining: 1.02s
547:	learn: 0.3305677	total: 1.23s	remaining: 1.01s
548:	learn: 0.3305556	total: 1.23s	remaining: 1.01s
549:	learn: 0.3304684	total: 1.23s	remaining: 1.01s
550:	learn: 0.3304450	total: 1.23s	remaining: 1s
551:	learn: 0.3301646	total: 1.24s	remaining: 1s
552:	learn: 0.3301148	total: 1.24s	remaining: 1s
553:	learn: 0.3299287	total: 1.24s	remaining: 998ms
554:	learn: 0.3297718	total: 1.24s	remaining: 996ms
555:	learn: 0.3296734	total: 1.24s	remaining: 993ms
556:	learn: 0.3295203	total: 1.25s	remaining: 991ms
557:	learn: 0.3294609	total: 1.25s	remaining: 988ms
558:	learn: 0.3292689	total: 1.25s	remaining: 986ms
559:	learn: 0.3291013	total: 1.25s	remaining: 984ms
560:	learn: 0.3290037	total: 1.25s	remaining: 982ms
561:	learn: 0.3288400	total: 1.26s	remaining: 979ms
562:	learn: 0.3286233	total: 1.26s	remaining: 977ms
563:	learn: 0.3285212	total: 1.26s	remaining: 974ms
564:	learn: 0.3284509	total: 1.26s	remaining: 972ms
565:	learn: 0.3284037

737:	learn: 0.3091751	total: 1.56s	remaining: 555ms
738:	learn: 0.3091493	total: 1.56s	remaining: 553ms
739:	learn: 0.3090366	total: 1.57s	remaining: 550ms
740:	learn: 0.3089280	total: 1.57s	remaining: 548ms
741:	learn: 0.3088014	total: 1.57s	remaining: 546ms
742:	learn: 0.3084586	total: 1.57s	remaining: 544ms
743:	learn: 0.3083734	total: 1.57s	remaining: 542ms
744:	learn: 0.3082807	total: 1.58s	remaining: 540ms
745:	learn: 0.3082458	total: 1.58s	remaining: 537ms
746:	learn: 0.3081274	total: 1.58s	remaining: 535ms
747:	learn: 0.3079663	total: 1.58s	remaining: 533ms
748:	learn: 0.3079123	total: 1.58s	remaining: 531ms
749:	learn: 0.3076756	total: 1.58s	remaining: 528ms
750:	learn: 0.3076710	total: 1.59s	remaining: 526ms
751:	learn: 0.3075965	total: 1.59s	remaining: 524ms
752:	learn: 0.3075177	total: 1.59s	remaining: 521ms
753:	learn: 0.3073854	total: 1.59s	remaining: 519ms
754:	learn: 0.3073545	total: 1.59s	remaining: 517ms
755:	learn: 0.3070440	total: 1.59s	remaining: 515ms
756:	learn: 

901:	learn: 0.2897065	total: 1.87s	remaining: 203ms
902:	learn: 0.2896279	total: 1.87s	remaining: 201ms
903:	learn: 0.2895873	total: 1.87s	remaining: 199ms
904:	learn: 0.2893651	total: 1.87s	remaining: 197ms
905:	learn: 0.2892498	total: 1.87s	remaining: 194ms
906:	learn: 0.2890693	total: 1.88s	remaining: 192ms
907:	learn: 0.2888373	total: 1.88s	remaining: 190ms
908:	learn: 0.2886210	total: 1.88s	remaining: 188ms
909:	learn: 0.2885223	total: 1.88s	remaining: 186ms
910:	learn: 0.2884594	total: 1.88s	remaining: 184ms
911:	learn: 0.2884277	total: 1.89s	remaining: 182ms
912:	learn: 0.2881080	total: 1.89s	remaining: 180ms
913:	learn: 0.2880343	total: 1.89s	remaining: 178ms
914:	learn: 0.2879636	total: 1.89s	remaining: 176ms
915:	learn: 0.2877047	total: 1.89s	remaining: 174ms
916:	learn: 0.2876302	total: 1.89s	remaining: 171ms
917:	learn: 0.2873985	total: 1.9s	remaining: 169ms
918:	learn: 0.2873215	total: 1.9s	remaining: 167ms
919:	learn: 0.2872336	total: 1.9s	remaining: 165ms
920:	learn: 0.2

,Model,Accuracy,Bal Acc.,Time
1,CatBoost,86.592179,84.769433,3.562472
2,XGBoost,84.916201,84.216074,0.477259
3,Skl GBM,84.357542,82.951252,1.236374
4,Skl HistGBM,84.357542,82.951252,0.743034
5,AdaBoost,83.798883,83.036891,0.185441
6,LightGBM,83.798883,82.496706,0.258912
7,Random Forest,82.681564,80.777339,0.215735
8,Extra Trees,79.888268,78.234519,0.211985
9,Decision Tree,75.977654,75.052701,0.656018


In [33]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [ ]:
"""
skf = model_selection.StratifiedKFold(
    # CODE HERE
)
"""
# YOUR CODE HERE
raise NotImplementedError()


results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""

# YOUR CODE HERE
raise NotImplementedError()


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

In [ ]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 7.1
Train with all data the best model

In [ ]:
# best_model = tree_classifiers["SELECT MY BEST MODEL HERE"]

# Fit best model with all data

# YOUR CODE HERE
raise NotImplementedError()

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [ ]:
# test_pred = # Get the predictions for x_test

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [ ]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

In [ ]:
sub.to_csv("sub.csv")

In [ ]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"